In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
warnings.filterwarnings('ignore')

In [2]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]

In [4]:
delimiters=["।","?","!"]

In [ ]:
fasttext=Word2Vec.load('fasttext/hi-d100-m2-fasttext.model')

In [ ]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [ ]:
def word_embedding(word):
    try:
        return fasttext.wv[word]
    except:
        return np.array([0.0]*100)

In [ ]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [ ]:
data=pd.read_csv('../hindi.csv')

# Sentence By Sentence

In [ ]:
verse_embeddings_sentence=[]
verse_embeddings_max=[]
verse_embeddings_mean=[]
for i in range(len(data)):
    text=data.loc[i]['Commentary']
    sentences=sent_tokenize(text)
    embeddings=[sentence_embedding(sentence) for sentence in sentences]
    #sentence
    verse_embeddings_sentence.append(embeddings)
    #Max Pooling
    norms=[lin.norm(i) for i in embeddings]
    index=norms.index(max(norms))
    verse_embeddings_max.append(embeddings[index])
    #Mean Pooling
    embeddings=np.array(list(map(lambda x: sum(x)/len(x), zip(*embeddings))))
    verse_embeddings_mean.append(embeddings)

# Whole

In [ ]:
verse_embeddings_whole=[]
for i in range(len(data)):
    text=data.loc[i]['Commentary']
    embeddings=sentence_embedding(text)
    verse_embeddings_whole.append(embeddings)

In [ ]:
file=open('sentence.pkl','wb')
pickle.dump(verse_embeddings_sentence,file)
file.close()
file=open('max.pkl','wb')
pickle.dump(verse_embeddings_max,file)
file.close()
file=open('mean.pkl','wb')
pickle.dump(verse_embeddings_mean,file)
file.close()

In [ ]:
file=open('whole.pkl','wb')
pickle.dump(verse_embeddings_whole,file)
file.close()